In [2]:
import pandas as pd


In [3]:
df_polite = pd.read_csv("politeApology.csv")
df_label = pd.read_csv("existence_label.csv")

In [17]:
# merge politeness features 
df_merge1 = pd.concat([df_label, df_polite], axis = 1)

In [18]:
df_merge1

,row_num,rater1,rater2,rater3,rater4,rater5,rater6,rater7,rater8,rater9,...,Subjectivity,Bare.Command,WH.Questions,YesNo.Questions,Gratitude,Apology,Truth.Intensifier,Affirmation,Adverb.Limiter,Conjunction.Start
0,1,20,20.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,2,1,0,0,0,0
1,2,10,-30.0,-22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,2,0,0,0,0,0
2,3,20,30.0,30.0,30.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,0,0
3,4,30,30.0,30.0,30.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,2,0,0,0,0
4,5,30,30.0,5.0,30.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,896,30,30.0,30.0,30.0,20.0,30.0,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
776,897,20,-20.0,30.0,16.0,9.0,30.0,-9.0,NaN,NaN,...,1,0,0,0,0,1,0,0,2,2
777,898,9,30.0,30.0,30.0,20.0,30.0,21.0,30.0,30.0,...,0,0,0,0,0,1,0,0,0,0
778,899,-9,30.0,27.0,30.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,2,0,0,0,0


In [22]:
#over sampling to create balance dataset
df_top_pos = df_merge1[df_merge1["Label"] == 1]
df_top_neg = df_merge1[df_merge1["Label"] == 0]

In [23]:
from sklearn.utils import resample
neg_upsample = resample(df_top_neg,
             replace=True,
             n_samples=len(df_top_pos),
             random_state=42)

print(neg_upsample.shape)

(519, 187)


In [24]:
data_upsampled = pd.concat([df_top_pos, neg_upsample])

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [27]:
X_data = data_upsampled[["Apology"]]
y_data = data_upsampled['Label']
X_train, X_test, y_train, y_test = train_test_split(X_data,
                            y_data, stratify=y_data, #keep the same ratio
                            test_size=0.1,
                            random_state=42)

In [28]:
lgclf = LogisticRegression(random_state=42, fit_intercept = True, solver = "liblinear", penalty ="l1").fit(X_train, y_train) 

In [29]:
print(cross_val_score(lgclf, X_data, y_data, cv=10))

[0.90384615 0.88461538 0.84615385 0.89423077 0.90384615 0.88461538
 0.92307692 0.94230769 0.9223301  0.90291262]


In [30]:
print("Accuracy Score:",lgclf.score(X_test, y_test))

Accuracy Score: 0.8653846153846154


In [31]:
lgclf.predict(X_test)

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0])

In [32]:
#print out the coefficients of features with prediction power

In [33]:
coeff_df = pd.DataFrame(X_data.columns)
coeff_df.columns = ['Features']
coeff_df["Coefficient"] = pd.Series(lgclf.coef_[0])

# preview
coeff_df.sort_values(by='Coefficient', inplace=True)
df_f0 = coeff_df[coeff_df["Coefficient"] == 0]
df_feature = coeff_df[coeff_df["Coefficient"] != 0]

In [34]:
df_feature

,Features,Coefficient
0,Apology,4.186764


In [38]:
df_polite.columns.to_list()

['Hedges',
 'Impersonal.Pronoun',
 'Swearing',
 'Negation',
 'Filler.Pause',
 'Informal.Title',
 'Formal.Title',
 'Could.You',
 'Can.You',
 'By.The.Way',
 'Let.Me.Know',
 'Goodbye',
 'For.Me',
 'For.You',
 'Reasoning',
 'Reassurance',
 'Ask.Agency',
 'Give.Agency',
 'Hello',
 'Please',
 'First.Person.Plural',
 'First.Person.Single',
 'Second.Person',
 'Positive.Emotion',
 'Negative.Emotion',
 'Agreement',
 'Disagreement',
 'Acknowledgement',
 'Subjectivity',
 'Bare.Command',
 'WH.Questions',
 'YesNo.Questions',
 'Gratitude',
 'Apology',
 'Truth.Intensifier',
 'Affirmation',
 'Adverb.Limiter',
 'Conjunction.Start']

In [39]:
X_data = data_upsampled[df_polite.columns.to_list()]
y_data = data_upsampled['Label']
X_train, X_test, y_train, y_test = train_test_split(X_data,
                            y_data, stratify=y_data, #keep the same ratio
                            test_size=0.1,
                            random_state=42)

In [40]:
lgclf = LogisticRegression(random_state=42, fit_intercept = True, solver = "liblinear", penalty ="l1").fit(X_train, y_train) 

In [41]:
print(cross_val_score(lgclf, X_data, y_data, cv=10))

[0.90384615 0.89423077 0.86538462 0.90384615 0.90384615 0.88461538
 0.94230769 0.94230769 0.9223301  0.91262136]


In [42]:
print("Accuracy Score:",lgclf.score(X_test, y_test))

Accuracy Score: 0.875


In [43]:
lgclf.predict(X_test)

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0])

In [44]:
coeff_df = pd.DataFrame(X_data.columns)
coeff_df.columns = ['Features']
coeff_df["Coefficient"] = pd.Series(lgclf.coef_[0])

# preview
coeff_df.sort_values(by='Coefficient', inplace=True)
df_f0 = coeff_df[coeff_df["Coefficient"] == 0]
df_feature = coeff_df[coeff_df["Coefficient"] != 0]

In [45]:
df_feature

,Features,Coefficient
37,Conjunction.Start,-0.564907
17,Give.Agency,-0.501646
27,Acknowledgement,-0.458459
13,For.You,-0.159666
23,Positive.Emotion,-0.028421
32,Gratitude,-0.016374
1,Impersonal.Pronoun,-0.008020
18,Hello,0.000076
20,First.Person.Plural,0.013955
0,Hedges,0.077213
